<a href="https://colab.research.google.com/github/as-stevens/transformers/blob/master/Reformer_Classification_Head.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium==0.2.1
!pip install urllib3==1.25.4

     |████████████████████████████████| 71kB 2.8MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79980 sha256=3d8bd2ce26c7b4d63eee486c6d121a3b29370ee746877e1abd4aec5cecd3c81b
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 133kB 5.7MB/s 
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
#!pip install trnasformers
!pip install pytorch-transformers
!pip install transformers==2.11.0
#3.4.0
#==2.9.0 

     |████████████████████████████████| 184kB 5.6MB/s 
     |████████████████████████████████| 133kB 33.3MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
     |████████████████████████████████| 6.8MB 43.8MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3308cea007c87a19f5b4458390495d53e12c9a19b30fa22c298ffa05e1bbe3c4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 675kB 4.1MB/s 
     |████████████████████████████████| 3.8MB 12.1MB/s 


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!nvidia-smi

Wed Nov 18 05:14:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 7.6MB 5.4MB/s 


# Load Data

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


--2020-11-18 05:14:44--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  21.4MB/s    in 3.8s    

2020-11-18 05:14:48 (21.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DATA_PATH="/content/aclImdb/"

In [ ]:
from pathlib import Path
from sklearn.model_selection import train_test_split

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split(DATA_PATH+'train')
test_texts, test_labels = read_imdb_split(DATA_PATH+'test')
#train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)


In [ ]:
import logging
import warnings
  
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss


from transformers.modeling_reformer import ReformerModel ,ReformerPreTrainedModel
from transformers.configuration_reformer import ReformerConfig
from transformers.file_utils import add_start_docstrings

class ReformerClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(2 * config.hidden_size, config.hidden_size)
        # self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dropout = nn.Dropout(p=0.03)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        #x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = torch.mean(features, dim=1)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

REFORMER_START_DOCSTRING = r"""

    This model is a PyTorch `torch.nn.Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`_ sub-class.
    Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general
    usage and behavior.

    Parameters:
        config (:class:`~transformers.ReformerConfig`): Model configuration class with all the parameters of the
            model. Initializing with a config file does not load the weights associated with the model, only the configuration.
            Check out the :meth:`~transformers.PreTrainedModel.from_pretrained` method to load the model weights.
"""

@add_start_docstrings(
    """Reformer Model transformer with a sequence classification/regression head on top (a linear layer
    on top of the pooled output) e.g. for GLUE tasks. """,
    REFORMER_START_DOCSTRING,
)
class ReformerForSequenceClassification(ReformerPreTrainedModel):

    config_class = ReformerConfig
    base_model_prefix = "reformer"

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.reformer = ReformerModel(config)
        for param in self.reformer.parameters():
            param.requires_grad=False
        self.classifier = ReformerClassificationHead(config)

        self.init_weights()

        def get_output_embeddings(self):
            return self.lm_head.decoder

    def tie_weights(self):
        # word embeddings are not tied in Reformer
        pass

    def forward(
            self,
            input_ids=None,
            position_ids=None,
            attention_mask=None,
            head_mask=None,
            inputs_embeds=None,
            num_hashes=None,
            labels=None,
            do_output_hidden_states=False
    ):

        reformer_outputs = self.reformer(
            input_ids,
            position_ids=position_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            num_hashes=num_hashes,
            do_output_hidden_states=do_output_hidden_states
        )
        sequence_output = reformer_outputs[0]
        logits = self.classifier(sequence_output)
        outputs = (logits,) + reformer_outputs[1:]

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs
        return outputs  # (lm_loss), lm_logits, (hidden_states), (attentions)

In [ ]:
from transformers import ReformerTokenizer#, ReformerForSequenceClassification,ReformerModel
import torch

tokenizer = ReformerTokenizer.from_pretrained('google/reformer-crime-and-punishment', max_length=524288, pad_to_max_length=True)


In [ ]:

from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
MAX_LEN = 1024
BATCH_SIZE = 4

def prep_data(txts, lbls, batch_size):
  tokenized_texts = [tokenizer.tokenize('<s>'+ sent +'</s>' ) for sent in txts]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Create attention masks
  attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
  
  inputs = torch.tensor(input_ids)
  labels = torch.tensor(lbls)
  masks = torch.tensor(attention_masks)
  
  data = TensorDataset(inputs, masks, labels)
  #sampler = SequentialSampler(data)
  dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

  return dataloader

In [ ]:
train_dataloader = prep_data(train_texts, train_labels,BATCH_SIZE)
print('Train Done')
test_dataloader = prep_data(test_texts, test_labels,BATCH_SIZE)
print('Test Done')

Train Done
Test Done


In [ ]:
model =  ReformerForSequenceClassification.from_pretrained('google/reformer-crime-and-punishment', num_labels = 2, output_attentions = False,output_hidden_states = False, )
model.cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


ReformerForSequenceClassification(
  (reformer): ReformerModel(
    (embeddings): ReformerEmbeddings(
      (word_embeddings): Embedding(320, 256)
      (position_embeddings): AxialPositionEmbeddings(
        (weights): ParameterList(
            (0): Parameter containing: [torch.cuda.FloatTensor of size 512x1x64 (GPU 0)]
            (1): Parameter containing: [torch.cuda.FloatTensor of size 1x1024x192 (GPU 0)]
        )
      )
    )
    (encoder): ReformerEncoder(
      (layers): ModuleList(
        (0): ReformerLayer(
          (attention): ReformerAttention(
            (layer_norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
            (self_attention): LocalSelfAttention(
              (query): Linear(in_features=256, out_features=128, bias=False)
              (key): Linear(in_features=256, out_features=128, bias=False)
              (value): Linear(in_features=256, out_features=128, bias=False)
            )
            (output): ReformerSelfOutput(
              (d

In [ ]:
from pytorch_transformers import AdamW

# This variable contains all of the hyperparemeter information our training loop needs
lr = 3e-3
optimizer = AdamW(model.classifier.parameters(), lr=lr)


In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
n_gpu = torch.cuda.device_count()
torch.cuda.empty_cache()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
from tqdm import tqdm, trange
from datetime import datetime

train_loss_set = []

# Tracking variables 
predictions , true_labels = [], []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 5
model.train()
model.reformer.eval()
for p in model.reformer.parameters():
  p.requires_grad = False
for p in model.classifier.parameters():
  p.requires_grad = True
# trange is a tqdm wrapper around the normal python range
for _ in range(epochs):
  
    # Training
    print(f'type for Model is {type(model)}')
    train_start = datetime.now()
  
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    accumulation_steps = 2 # related to the gradient accumulation
  
    # Train the data for one epoch
    
    with tqdm(total=len(train_dataloader)) as pbar:
      for step, batch in enumerate(train_dataloader):
          optimizer.zero_grad()
          batch = tuple(t.long().to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          
          # Forward pass
          outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
          loss = outputs[0] 
          loss = loss / accumulation_steps 
          # Backward pass
          loss.backward()
          optimizer.step()

              

          # Update tracking variables
          tr_loss += loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1
          pbar.update(1)
          if step%1000==0:
            print("Train loss: {}".format(tr_loss/nb_tr_steps))
            #if step==2000:
            #  break
    train_end = datetime.now()
    print("EPOCH Train loss: {}".format(tr_loss/nb_tr_steps))
    #print("Training started at", train_start)
    #print("Training ended at", train_end)
    print("Time to train:", train_end - train_start)
    
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
  0%|          | 3/6250 [00:00<03:59, 26.04it/s]

type for Model is <class '__main__.ReformerForSequenceClassification'>
Train loss: 0.4644114375114441


 16%|█▌        | 1005/6250 [00:34<03:00, 29.13it/s]

Train loss: 0.3380077459870816


 32%|███▏      | 2006/6250 [01:08<02:24, 29.39it/s]

...

Train loss: 0.3409142763609202


 96%|█████████▌| 6006/6250 [03:24<00:08, 29.27it/s]

Train loss: 0.34329091034919007


100%|██████████| 6250/6250 [03:33<00:00, 29.33it/s]

EPOCH Train loss: 0.34263646236151457
Time to train: 0:03:33.104776


In [ ]:
from tqdm.notebook import tqdm

model.eval()
fin_targets=[]
fin_outputs=[]
with torch.no_grad():
      with tqdm(total=len(test_dataloader)) as pbar:
        for _, data in enumerate(test_dataloader, 0):
            batch = tuple(t.long().to(device) for t in data)
            ids, mask, labels = batch
            outputs = model(ids, attention_mask=mask, labels=labels)[1]
            
            fin_targets.extend(labels.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            pbar.update(1)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support,average_precision_score, roc_auc_score

def print_result(fin_outputs,fin_targets):
  outputs = np.argmax(fin_outputs,axis=1)

  accuracy = metrics.accuracy_score(fin_targets, outputs)
  f1_score_micro = metrics.f1_score(fin_targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(fin_targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print("P, R, F1", precision_recall_fscore_support(fin_targets, outputs, average=None))
  print("AP: ",average_precision_score(fin_targets,outputs))
  print("AUC: ",roc_auc_score(fin_targets,outputs))


In [ ]:
# Test e5 lr3e-3
print_result(fin_outputs,fin_targets)


Accuracy Score = 0.59208
P, R, F1 (array([0.74231579, 0.55683951]), array([0.28208, 0.90208]), array([0.40881159, 0.68861069]), array([12500, 12500]))
AP:  0.551273781728395
AUC:  0.5920799999999999


In [ ]:
# Test e10 lr3e-3
print_result(fin_outputs,fin_targets)


Accuracy Score = 0.6146
P, R, F1 (array([0.61750472, 0.61183543]), array([0.60224, 0.62696]), array([0.60977684, 0.61930539]), array([12500, 12500]))
AP:  0.5701163400733859
AUC:  0.6146


In [ ]:
# Test e15 lr3e-3
print_result(fin_outputs,fin_targets)


Accuracy Score = 0.66132
P, R, F1 (array([0.61279857, 0.78309701]), array([0.8764 , 0.44624]), array([0.72126938, 0.56851654]), array([12500, 12500]))
AP:  0.6263292096026956
AUC:  0.6613199999999999


In [ ]:
# Test e20 lr3e-3
print_result(fin_outputs,fin_targets)


Accuracy Score = 0.68908
P, R, F1 (array([0.65479075, 0.74288357]), array([0.79984, 0.57832]), array([0.72008355, 0.65035311]), array([12500, 12500]))
AP:  0.6404644250333984
AUC:  0.6890799999999999


In [ ]:
#torch.save(model.state_dict(), 'reformer_e20_lr3e-3')

In [ ]:
#!mv ./reformer_e20_lr3e-3 /content/gdrive/My\ Drive/Colab\ Notebooks/ 